<a href="https://colab.research.google.com/github/DenisseJ/Proyecto_Final/blob/main/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final

- Se instalan e importan las librerías necesarias

In [1]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 28, in <module>
    if ('K80' not in gpu_name):
TypeError: a bytes-like object is required, not 'str'


In [11]:
!pip install gradio

In [14]:
!pip install --upgrade typing-extensions

In [9]:
!pip install gradio --upgrade

In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import locale

- Se accede a la base de datos



In [2]:
!wget https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

--2023-11-12 23:41:25--  https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 201.98.60.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|201.98.60.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18045440 (17M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip’

datos_abiertos_covi 100%[===================>]  17.21M  8.04MB/s    in 2.1s    

2023-11-12 23:41:27 (8.04 MB/s) - ‘datos_abiertos_covid19.zip’ saved [18045440/18045440]



In [3]:
!unzip /content/datos_abiertos_covid19.zip

Archive:  /content/datos_abiertos_covid19.zip
  inflating: COVID19MEXICO.csv       


In [4]:
%load_ext cudf.pandas

In [17]:
data = pd.read_csv("COVID19MEXICO.csv", low_memory = False)
data.columns

StringIndex(['FECHA_ACTUALIZACION' 'ID_REGISTRO' 'ORIGEN' 'SECTOR' 'ENTIDAD_UM' 'SEXO'
 'ENTIDAD_NAC' 'ENTIDAD_RES' 'MUNICIPIO_RES' 'TIPO_PACIENTE'
 'FECHA_INGRESO' 'FECHA_SINTOMAS' 'FECHA_DEF' 'INTUBADO' 'NEUMONIA' 'EDAD'
 'NACIONALIDAD' 'EMBARAZO' 'HABLA_LENGUA_INDIG' 'INDIGENA' 'DIABETES'
 'EPOC' 'ASMA' 'INMUSUPR' 'HIPERTENSION' 'OTRA_COM' 'CARDIOVASCULAR'
 'OBESIDAD' 'RENAL_CRONICA' 'TABAQUISMO' 'OTRO_CASO' 'TOMA_MUESTRA_LAB'
 'RESULTADO_LAB' 'TOMA_MUESTRA_ANTIGENO' 'RESULTADO_ANTIGENO'
 'CLASIFICACION_FINAL' 'MIGRANTE' 'PAIS_NACIONALIDAD' 'PAIS_ORIGEN' 'UCI'], dtype='object')

In [19]:
data.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1186694 entries, 0 to 1186693
Data columns (total 40 columns):
 #   Column                 Non-Null Count    Dtype
---  ------                 --------------    -----
 0   FECHA_ACTUALIZACION    1186694 non-null  object
 1   ID_REGISTRO            1186694 non-null  object
 2   ORIGEN                 1186694 non-null  int64
 3   SECTOR                 1186694 non-null  int64
 4   ENTIDAD_UM             1186694 non-null  int64
 5   SEXO                   1186694 non-null  int64
 6   ENTIDAD_NAC            1186694 non-null  int64
 7   ENTIDAD_RES            1186694 non-null  int64
 8   MUNICIPIO_RES          1186694 non-null  int64
 9   TIPO_PACIENTE          1186694 non-null  int64
 10  FECHA_INGRESO          1186694 non-null  object
 11  FECHA_SINTOMAS         1186694 non-null  object
 12  FECHA_DEF              1186694 non-null  object
 13  INTUBADO               1186694 non-null  int64
 14  NEUMONIA               1186694 non-null  in

In [20]:
data.describe()

/usr/local/lib/python3.10/dist-packages/cudf/core/dataframe.py:5062: FutureWarning: `datetime_is_numeric` is deprecated. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/cudf/core/series.py:3334: FutureWarning: `datetime_is_numeric` is deprecated and will be removed in a future release. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(


,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,INTUBADO,NEUMONIA,...,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,UCI
count,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,...,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06,1.186694e+06
mean,1.692639e+00,7.030433e+00,1.453498e+01,1.411661e+00,1.534343e+01,1.471309e+01,3.856584e+01,1.083214e+00,8.911062e+01,2.440521e+00,...,2.076030e+00,2.051011e+00,3.407123e+00,1.806609e+00,7.861505e+01,1.145288e+00,1.547294e+01,5.487269e+00,9.866122e+01,8.911151e+01
std,4.614000e-01,3.830850e+00,7.798240e+00,4.921350e-01,9.505631e+00,7.762377e+00,7.497106e+01,2.762050e-01,2.622312e+01,6.757076e+00,...,2.897062e+00,2.974038e+00,1.215742e+01,3.949570e-01,3.754819e+01,3.523910e-01,3.361594e+01,1.916392e+00,5.734053e+00,2.622033e+01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,1.000000e+00,4.000000e+00,9.000000e+00,1.000000e+00,9.000000e+00,9.000000e+00,7.000000e+00,1.000000e+00,9.700000e+01,2.000000e+00,...,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,9.700000e+01,1.000000e+00,1.000000e+00,3.000000e+00,9.900000e+01,9.700000e+01
50%,2.000000e+00,4.000000e+00,1.300000e+01,1.000000e+00,1.400000e+01,1.400000e+01,1.700000e+01,1.000000e+00,9.700000e+01,2.000000e+00,...,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,9.700000e+01,1.000000e+00,2.000000e+00,7.000000e+00,9.900000e+01,9.700000e+01
75%,2.000000e+00,1.200000e+01,2.000000e+01,2.000000e+00,2.100000e+01,2.000000e+01,4.400000e+01,1.000000e+00,9.700000e+01,2.000000e+00,...,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,9.700000e+01,1.000000e+00,2.000000e+00,7.000000e+00,9.900000e+01,9.700000e+01
max,2.000000e+00,9.900000e+01,3.200000e+01,2.000000e+00,9.900000e+01,3.200000e+01,9.990000e+02,2.000000e+00,9.900000e+01,9.900000e+01,...,9.800000e+01,9.800000e+01,9.900000e+01,2.000000e+00,9.700000e+01,2.000000e+00,9.700000e+01,7.000000e+00,9.900000e+01,9.900000e+01


Generamos copia de los datos

In [21]:
df = data
df

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2023-11-07,0ed066,2,12,16,2,16,16,41,1,...,2,2,97,1,2,7,99,México,97,97
1,2023-11-07,6cef7b,2,12,7,1,7,7,108,1,...,2,2,97,2,97,6,99,México,97,97
2,2023-11-07,e72a0b,1,12,21,1,21,21,140,1,...,1,2,97,1,1,3,99,México,97,97
3,2023-11-07,dcee4a,1,6,11,1,11,11,20,1,...,1,2,97,1,1,3,99,México,97,97
4,2023-11-07,6236e4,2,12,9,2,9,9,9,1,...,1,2,97,1,1,3,99,México,97,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186689,2023-11-07,m00ee10,2,12,15,2,30,15,33,1,...,99,2,97,1,2,7,99,México,97,97
1186690,2023-11-07,m1cc581,2,3,15,1,20,15,999,1,...,99,2,97,1,2,7,99,México,97,97
1186691,2023-11-07,m0b0348,2,12,15,1,9,15,999,1,...,99,2,97,1,2,7,99,México,97,97
1186692,2023-11-07,m08f43c,2,3,15,2,15,15,999,1,...,99,2,97,1,2,7,99,México,97,97


Las variables a considerar para determinar si el paciente padece covid, es sospechoso contagio o no padece covid, incluyen datos relacionados a la salid del paciente solamente.
Por lo tanto, se eliminan las columnas de vivienda, origenes y habla.

In [22]:
df = df.drop( columns = ["SECTOR", "MUNICIPIO_RES", "HABLA_LENGUA_INDIG", "ID_REGISTRO", "ORIGEN", "PAIS_NACIONALIDAD", "PAIS_ORIGEN", "INDIGENA", "ENTIDAD_UM", "ENTIDAD_NAC", "NACIONALIDAD", "ENTIDAD_RES"])


Para identificar si se trata de un caso de covid confirmado, sospechoso o negativo se utilizó la columns CLASIFICACIÓN_FINAL. Donde los valores 1, 2 y 3, corresponden a casos confimados. El 4 es inválido, el 5 no realizado por laboratorio. El valor 5 es asignado a los casos sospechosos y el 6 a los negativos.

In [23]:
df['CLASIFICACION_FINAL']

0          7
1          6
2          3
3          3
4          3
          ..
1186689    7
1186690    7
1186691    7
1186692    7
1186693    7
Name: CLASIFICACION_FINAL, Length: 1186694, dtype: int64

Se identifica la cantidad de pacientes en cada categoría

In [24]:
df['CLASIFICACION_FINAL'].value_counts()

7    692424
3    419475
6     64459
1      7969
5      2177
4       164
2        26
Name: CLASIFICACION_FINAL, dtype: int64

Los valores con menos cantidad fueron los de 5, 4 y 2. Estos no influyen en el estado de la persona, ya que se trata de casos en los que no se proceso la muestra, esta resultó no válida o fue confirmado por dictaminación debido a la invalidez de la muestra.
Por ello no son útiles en el modelo y son descartados.

In [25]:
df=df.drop(df[df['CLASIFICACION_FINAL']==2].index)
df=df.drop(df[df['CLASIFICACION_FINAL']==4].index)
df=df.drop(df[df['CLASIFICACION_FINAL']==5].index)

In [26]:
df['CLASIFICACION_FINAL'].replace({3: 2, 6: 3, 7: 4}, inplace=True)

In [27]:
df['CLASIFICACION_FINAL'].value_counts()

4    692424
2    419475
3     64459
1      7969
Name: CLASIFICACION_FINAL, dtype: int64

Cambiamos el formato de datos con fecha

In [28]:
df['FECHA_ACTUALIZACION'] = pd.to_datetime(df['FECHA_ACTUALIZACION'])
df['FECHA_INGRESO'] = pd.to_datetime(df['FECHA_INGRESO'])
df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'])
df['FECHA_DEF'] = pd.to_datetime(df['FECHA_DEF'],errors='coerce')

Se crean las columnas de días de defunción, hospitalización y sintomas a apatir de las fechas.

In [29]:
df['DIAS_DEFUNCION']=(df['FECHA_ACTUALIZACION']- df['FECHA_DEF']).dt.days
df['DIAS_HOSPITALIZACION']=(df['FECHA_ACTUALIZACION']- df['FECHA_INGRESO']).dt.days
df['DIAS_SINTOMAS']=(df['FECHA_ACTUALIZACION']- df['FECHA_SINTOMAS']).dt.days


In [30]:
df["DIAS_DEFUNCION"] = df["DIAS_DEFUNCION"].fillna(0)

In [31]:
df['DIAS_DEFUNCION']

0          0
1          0
2          0
3          0
4          0
          ..
1186689    0
1186690    0
1186691    0
1186692    0
1186693    0
Name: DIAS_DEFUNCION, Length: 1184327, dtype: int64

In [32]:
df = df.drop( columns = ['FECHA_ACTUALIZACION', 'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF'])

Modificamos el tipo de columnas

In [33]:
for col in ['TIPO_PACIENTE']:
  df[col]=df[col].astype(str)

In [34]:
le = LabelEncoder()
for col in ['TIPO_PACIENTE','SEXO']:
  df[col] = le.fit_transform(df[col])

Asignamos la variable dependiente e independiente

In [35]:
X = df.drop(columns=['CLASIFICACION_FINAL'])
y = df['CLASIFICACION_FINAL']

Establecemos el conjunto de datos de entrenamiento y validación.

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

- Usando sklearn

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [38]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.16      0.04      0.06      1579
           2       1.00      1.00      1.00     83925
           3       0.97      0.97      0.97     13017
           4       0.99      1.00      0.99    138345

    accuracy                           0.99    236866
   macro avg       0.78      0.75      0.76    236866
weighted avg       0.99      0.99      0.99    236866



Validación cruzada

In [39]:
scores = cross_val_score(clf, X, y, cv=6)
print("Puntajes:", scores)
print("Puntuación media:", scores.mean())

Puntajes: [0.98445194 0.97294668 0.96564634 0.96116279 0.97230835 0.92510145]
Puntuación media: 0.9636029251199418


- Usando cuml

In [42]:
import cuml
from cuml.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf=RandomForestClassifier(random_state=42)
X_train=X_train.astype('float32')
y_train=y_train.astype('float32')
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)


/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


In [44]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.76      0.01      0.02      1579
           2       1.00      1.00      1.00     83925
           3       0.97      0.97      0.97     13017
           4       0.99      1.00      0.99    138345

    accuracy                           0.99    236866
   macro avg       0.93      0.75      0.75    236866
weighted avg       0.99      0.99      0.99    236866



- Comparación con KNC


In [50]:
import cuml
from cuml.ensemble import RandomForestClassifier
from cuml.neighbors import KNeighborsClassifier


cuml_RFC=RandomForestClassifier(random_state=42)
cuml_KNC=KNeighborsClassifier()

classifiers=[
    {
        'Ramdom Forest',cuml_RFC
    },
    {
        'KNN',cuml_RFC
    }
]

X_train=X_train.astype('float32')
y_train=y_train.astype('float32')

for name,clf in classifiers:
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print(f'Modelo resultados {name}')
  print(classification_report(y_test,y_pred.get()))

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


Modelo resultados Ramdom Forest
              precision    recall  f1-score   support

           1       0.76      0.01      0.02      1579
           2       1.00      1.00      1.00     83925
           3       0.97      0.97      0.97     13017
           4       0.99      1.00      0.99    138345

    accuracy                           0.99    236866
   macro avg       0.93      0.75      0.75    236866
weighted avg       0.99      0.99      0.99    236866

Modelo resultados KNN
              precision    recall  f1-score   support

           1       0.76      0.01      0.02      1579
           2       1.00      1.00      1.00     83925
           3       0.97      0.97      0.97     13017
           4       0.99      1.00      0.99    138345

    accuracy                           0.99    236866
   macro avg       0.93      0.75      0.75    236866
weighted avg       0.99      0.99      0.99    236866



## Interfaz de gradio

In [52]:
import locale
def getpreferredencoding(do_setlocale = True):
  return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [53]:
import gradio as gr
import pandas as pd
import numpy as np


def predict(sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
             diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno,
           uci, dias_hospitalizacion, dias_sintomas):


    # Crea un DataFrame con los valores de entrada
    input_data = pd.DataFrame([[
        sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
       diabetes, epoc, asma, inmusupr,
        hipertension, otra_com, cardiovascular, obesidad,
        renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
        resultado_lab, toma_muestra_antigeno, resultado_antigeno, uci, dias_hospitalizacion, dias_sintomas
        ]], columns=[
        'SEXO', 'TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'EDAD', 'EMBARAZO','DIABETES', 'EPOC',
        'ASMA', 'INMUSUPR','HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD','RENAL_CRONICA',
        'TABAQUISMO', 'OTRO_CASO', 'TOMA_MUESTRA_LAB','RESULTADO_LAB', 'TOMA_MUESTRA_ANTIGENO',
        'RESULTADO_ANTIGENO','UCI', 'DIAS_HOSPITALIZACION', 'DIAS_SINTOMAS'])

    prediction = clf.predict(input_data)
    return prediction




gr.Row
inputs = [
    gr.Radio(choices=[0, 1], label='Sexo'),
    gr.Radio(choices=[0, 1], label='Tipo de Paciente'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Intubado'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Neumonía'),
    gr.Slider(minimum=0, maximum=120, value=30, label='Edad'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Embarazo'),
    gr.Radio(choices=[1, 2, 98], label='Diabetes'),
    gr.Radio(choices=[1, 2, 98], label='EPOC'),
    gr.Radio(choices=[1, 2, 98], label='Asma'),
    gr.Radio(choices=[1, 2, 98], label='Inmunosupresión'),
    gr.Radio(choices=[1, 2, 98], label='Hipertensión'),
    gr.Radio(choices=[1, 2, 98], label='Otra Comorbilidad'),
    gr.Radio(choices=[1, 2, 98], label='Cardiovascular'),
    gr.Radio(choices=[1, 2, 98], label='Obesidad'),
    gr.Radio(choices=[1, 2, 98], label='Enfermedad Renal Crónica'),
    gr.Radio(choices=[1, 2, 98], label='Tabaquismo'),
    gr.Radio(choices=[1, 2, 99], label='Contacto con otro caso'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de laboratorio'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de laboratorio'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='UCI'),
    gr.Number(label='Días de Hospitalización',value=0),
    gr.Number(label='Días con Síntomas',value=0),
]

# Crear componente de salida para Gradio
outputs = gr.Textbox(label="Predicción")

# Crear la interfaz de Gradio
demo = gr.Interface(fn=predict, inputs=inputs, outputs=outputs)

# Ejecutar la aplicación web
if __name__ == "__main__":
    demo.launch(show_api=False,debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://21cf0570ad8e5df43c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://21cf0570ad8e5df43c.gradio.live
